In [1]:
import math
import random

import pandas as pd
import transformers
from datasets import ClassLabel, load_dataset
from IPython.display import HTML, display
from transformers import (
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

In [2]:
dataset_name = "uestc-swahili/swahili"
datasets = load_dataset(dataset_name)

In [3]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 42069
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3371
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3372
    })
})

In [4]:
datasets["train"][0]

{'text': 'taarifa hiyo ilisema kuwa ongezeko la joto la maji juu ya wastani katikati ya bahari ya UNK inaashiria kuwepo kwa mvua za el nino UNK hadi mwishoni mwa april ishirini moja sifuri imeelezwa kuwa ongezeko la joto magharibi mwa bahari ya hindi linatarajiwa kuhamia katikati ya bahari hiyo hali ambayo itasababisha pepo kutoka kaskazini mashariki kuvuma kuelekea bahari ya hindi'}

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


show_random_elements(datasets["train"])

,text
0,watakuja ili kumuangalia wanataka kumuona akicheza mechi ngumu
1,anafafanua kuwa kama tanzania itajipanga vizuri ina uwezo wa kulisha wananchi wake na nchi zote jirani kwani ina rasilimali ya maji ya maziwa mito ya kutosha na ardhi yenye rutuba nzuri
2,anasema endapo kura za kuanzisha jopo hilo UNK atachukua jukumu la kusikiliza madai hayo akiamini kuwa wadau wengine watafanya hivyo ili isije kutokea tena
3,maagizo ya rais UNK kesi kwani utekelezaji wa kuwasha mitambo hiyo unapitia kwa msimamizi wa muda aliyeteuliwa na mahakama
4,akizunguzia suala la mauaji ya albino rais kikwete alisema kuwa ongezeko la mauaji hayo linatokana na imani za kishirikina
5,hakuna utafiti wa kisayansi UNK kwamba dawa hizo zinasababisha saratani watu hao ni waongo na UNK na jamii kwa kuwa UNK na umasikini wa fikra tena mawazo mgando alisema na kuongeza kuwa moja ya faida ya uzazi ya uzazi wa mpango ni kuisadia familia kupanga shughuli za maendeleo ikiwemo kurudisha afya ya mama kabla ya kubeba mimba
6,ni vitu vigeni ambavyo wanatakiwa kukutana navyo mara nyingine tena ili UNK
7,hili UNK wengi kutokana dharau ya wazi ya mkapa UNK kwa waandishi wa tanzania na kuwathamini waandishi wa nchi za magharibi
8,ferguson amefanikiwa kusajili wachezaji wa nje pia england kwa UNK
9,hicho ndio ninachotaka alisema


In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"])


model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["text"]
)

/opt/anaconda3/envs/development/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 42069
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3371
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3372
    })
})

In [8]:
tokenized_datasets["train"][10]

{'input_ids': [0,
  150722,
  151,
  12967,
  151,
  149362,
  10790,
  6,
  180323,
  177769,
  228,
  186214,
  28803,
  6,
  180323,
  228,
  85407,
  14102,
  24,
  32825,
  75,
  68428,
  10475,
  760,
  291,
  4390,
  4542,
  12209,
  686,
  259,
  130719,
  1499,
  92704,
  291,
  1783,
  157,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [9]:
show_random_elements(tokenized_datasets["train"])

,input_ids,attention_mask
0,"[0, 56038, 11, 68163, 12116, 3877, 210909, 1922, 41144, 92289, 23317, 20973, 13037, 51795, 210131, 70037, 21, 73365, 291, 43319, 151, 92368, 760, 23103, 151, 1922, 57942, 39665, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"[0, 15018, 15800, 370, 213001, 31851, 75, 68428, 41002, 760, 6, 48949, 11412, 21, 91750, 2197, 45725, 201451, 35466, 24, 38765, 148301, 1499, 139470, 75, 145, 31, 30601, 151, 2285, 182828, 10219, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,"[0, 876, 24086, 242113, 153130, 259, 1436, 63220, 14, 259, 75206, 634, 259, 83408, 1622, 123490, 14, 300, 36593, 142637, 22476, 166574, 6, 180323, 26542, 38388, 11, 3125, 59428, 90965, 4848, 2959, 24, 135637, 24, 17163, 14, 24258, 18333, 300, 147119, 760, 42323, 20439, 279, 57268, 12787, 26542, 93, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,"[0, 347, 169573, 259, 180486, 19562, 56222, 111263, 6, 180323, 153130, 70768, 43744, 5431, 63766, 229178, 151, 1922, 186214, 30714, 63549, 10744, 21428, 147555, 72543, 55617, 24, 97804, 1922, 21528, 11, 1622, 188197, 1608, 41217, 151, 57055, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,"[0, 7975, 23023, 38424, 105177, 10, 33169, 88043, 5538, 184037, 23, 21256, 6, 180323, 3886, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,"[0, 177537, 6, 180323, 13487, 24, 466, 34, 120209, 760, 200189, 80, 203463, 23, 21256, 9873, 172065, 81760, 223090, 56070, 50297, 185840, 24, 200189, 19712, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
6,"[0, 169182, 259, 43879, 3597, 200, 44910, 24, 700, 596, 10391, 133, 1622, 3441, 151, 124, 33970, 74667, 6, 180323, 228, 14336, 634, 21, 101950, 7126, 18285, 113824, 259, 52606, 763, 11, 75, 61166, 75, 196654, 86286, 80, 1257, 1352, 658, 1622, 51683, 26045, 2224, 1922, 34641, 197930, 80, 809, 3390, 151, 167892, 24, 58606, 31, 26045, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
7,"[0, 18285, 237, 8380, 24, 57879, 79, 771, 8767, 50032, 4571, 50397, 17, 771, 13598, 4522, 15026, 78, 22455, 259, 228, 11222, 56992, 11, 178235, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
8,"[0, 6, 135187, 16115, 291, 58187, 95491, 19562, 1922, 300, 14102, 24, 228, 2285, 33265, 51368, 5748, 2224, 6, 100352, 200, 33226, 1608, 347, 195209, 259, 61131, 14, 6758, 10790, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
9,"[0, 77420, 259, 119, 11, 71596, 238, 115, 1462, 20205, 400, 1436, 151, 228, 139294, 18285, 188197, 145211, 24222, 110867, 24, 6, 180323, 1072, 416, 151, 72543, 9908, 24, 81760, 259, 18398, 316, 17540, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [10]:
import math

print(f"Model maximum block size: {tokenizer.model_max_length}")
# block_size = tokenizer.model_max_length
block_size = 128
batch_size = 1000
print(f"Block size: {block_size}")

for type in ["train", "test", "validation"]:
    num_tokens = sum(
        [len(input_ids) for input_ids in tokenized_datasets[type]["input_ids"]]
    )
    print(
        f"The number of tokens in {type}: {num_tokens}, this will be ~{round(num_tokens / block_size)} blocks."
    )

Model maximum block size: 512
Block size: 128
The number of tokens in train: 1865542, this will be ~14575 blocks.
The number of tokens in test: 166669, this will be ~1302 blocks.
The number of tokens in validation: 159984, this will be ~1250 blocks.


In [11]:
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [12]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [13]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14552
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1301
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1248
    })
})

In [14]:
show_random_elements(lm_datasets["train"])

,input_ids,attention_mask,labels
0,"[180323, 173, 192, 179616, 151, 2197, 45725, 2, 0, 20205, 234538, 151, 180486, 19562, 151, 134375, 173, 192, 24, 10805, 24205, 12926, 156300, 18333, 1337, 145, 206068, 137566, 300, 1053, 433, 23210, 151, 83, 50637, 93, 24, 33780, 151, 2197, 45725, 2626, 2, 0, 923, 913, 145, 156300, 5748, 300, 68524, 14, 347, 18, 63991, 100484, 7502, 11, 156, 28783, 5941, 2514, 200, 21045, 1648, 516, 308, 27651, 200, 519, 192, 24, 3114, 10521, 24, 10805, 156300, 99477, 5431, 206068, 137566, 180486, 19562, 2, 0, 20205, 180486, 19562, 6, 180323, 24, 52041, 45046, 259, 14378, 9353, 6, 180323, 94194, 24258, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[180323, 173, 192, 179616, 151, 2197, 45725, 2, 0, 20205, 234538, 151, 180486, 19562, 151, 134375, 173, 192, 24, 10805, 24205, 12926, 156300, 18333, 1337, 145, 206068, 137566, 300, 1053, 433, 23210, 151, 83, 50637, 93, 24, 33780, 151, 2197, 45725, 2626, 2, 0, 923, 913, 145, 156300, 5748, 300, 68524, 14, 347, 18, 63991, 100484, 7502, 11, 156, 28783, 5941, 2514, 200, 21045, 1648, 516, 308, 27651, 200, 519, 192, 24, 3114, 10521, 24, 10805, 156300, 99477, 5431, 206068, 137566, 180486, 19562, 2, 0, 20205, 180486, 19562, 6, 180323, 24, 52041, 45046, 259, 14378, 9353, 6, 180323, 94194, 24258, ...]"
1,"[22539, 1681, 347, 35251, 259, 11625, 20646, 6544, 432, 21, 146904, 11, 142636, 347, 18357, 35251, 259, 23546, 259, 8840, 1556, 1462, 29970, 142, 10744, 242193, 2420, 24, 61192, 151, 11625, 31, 2, 0, 104697, 93, 24205, 135382, 30938, 300, 5614, 42753, 979, 1053, 433, 370, 84641, 108553, 18802, 143, 12781, 1622, 104697, 151, 34912, 350, 24, 129543, 151, 279, 3584, 49270, 300, 9382, 83, 50637, 93, 24, 308, 157, 5431, 210832, 24, 2223, 26770, 324, 50742, 2223, 1681, 5485, 11230, 24, 154643, 3262, 95560, 2, 0, 347, 3767, 95563, 259, 25988, 1622, 51368, 151, 139803, 151, 224369, 259, 170712, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[22539, 1681, 347, 35251, 259, 11625, 20646, 6544, 432, 21, 146904, 11, 142636, 347, 18357, 35251, 259, 23546, 259, 8840, 1556, 1462, 29970, 142, 10744, 242193, 2420, 24, 61192, 151, 11625, 31, 2, 0, 104697, 93, 24205, 135382, 30938, 300, 5614, 42753, 979, 1053, 433, 370, 84641, 108553, 18802, 143, 12781, 1622, 104697, 151, 34912, 350, 24, 129543, 151, 279, 3584, 49270, 300, 9382, 83, 50637, 93, 24, 308, 157, 5431, 210832, 24, 2223, 26770, 324, 50742, 2223, 1681, 5485, 11230, 24, 154643, 3262, 95560, 2, 0, 347, 3767, 95563, 259, 25988, 1622, 51368, 151, 139803, 151, 224369, 259, 170712, ...]"
2,"[50350, 7180, 137365, 760, 13598, 9873, 53229, 7933, 763, 2057, 5234, 18285, 15018, 13598, 7431, 4571, 52327, 32825, 129543, 5748, 760, 1922, 23317, 23, 2832, 11, 38435, 8426, 373, 107, 171, 964, 33778, 760, 23541, 6, 180323, 760, 148, 22242, 24, 228, 104601, 48, 561, 1499, 228, 33, 15586, 24, 173975, 12967, 172, 151, 218761, 153130, 2, 0, 259, 104601, 658, 259, 75206, 634, 259, 48, 561, 300, 70521, 34721, 8971, 771, 21383, 118468, 1158, 96713, 7126, 6, 180323, 35466, 24, 173975, 151, 104697, 24258, 80, 115130, 24, 23317, 2, 0, 177391, 151, 76937, 24, 91630, 259, 49157, 17, 139818, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[50350, 7180, 137365, 760, 13598, 9873

In [15]:
model_checkpoint = "xlm-roberta-base"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
print(
    f"{model_checkpoint} number of parameters: {round(model.num_parameters() / 1_000_000)}M"
)

xlm-roberta-base number of parameters: 278M


In [17]:
training_args = TrainingArguments(
    f"{model_checkpoint}-finetuned-{dataset_name.split('/')[-1]}",
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [18]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [19]:
small_train_dataset = lm_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = lm_datasets["validation"].shuffle(seed=42).select(range(1000))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
)

In [20]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/125 [00:00<?, ?it/s]

>>> Perplexity: 24.25


In [21]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.0673115253448486, 'eval_model_preparation_time': 0.0016, 'eval_runtime': 27.8459, 'eval_samples_per_second': 35.912, 'eval_steps_per_second': 4.489, 'epoch': 1.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.0007805824279785, 'eval_model_preparation_time': 0.0016, 'eval_runtime': 27.5371, 'eval_samples_per_second': 36.315, 'eval_steps_per_second': 4.539, 'epoch': 2.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.9776479005813599, 'eval_model_preparation_time': 0.0016, 'eval_runtime': 28.2579, 'eval_samples_per_second': 35.388, 'eval_steps_per_second': 4.424, 'epoch': 3.0}
{'train_runtime': 1192.1127, 'train_samples_per_second': 2.517, 'train_steps_per_second': 0.315, 'train_loss': 2.5689215494791666, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=2.5689215494791666, metrics={'train_runtime': 1192.1127, 'train_samples_per_second': 2.517, 'train_steps_per_second': 0.315, 'total_flos': 197909291520000.0, 'train_loss': 2.5689215494791666, 'epoch': 3.0})

In [22]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/125 [00:00<?, ?it/s]

>>> Perplexity: 7.06


In [23]:
trainer.save_model("./my_model")